In [0]:
## To access my google drive where I stored all the python files and will save the results

from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/My Drive/dead_neuron_emp_3')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import os
from load_data_function import load_dataset
from network_function import Network
#from dead_node_function import check_zero_nodes, register_node_hooks
from train_network_function import train_network
import pickle
from misc_function import create_folder, write_info_txt, create_net_dict, calc_rank
#from opt_dead_nodes_function import opt_dead_nodes
#from rank_function import convex_comb_rank
import torch
#from volume_function import calc_vol
from shuffled_label_function import get_random_loader
#import random

save_folder_created = False

NET_dict = create_net_dict() ## The neural network parameters are created here
current_net_gen = NET_dict.keys()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

### The main parameters are here
use_random_labels = False ## Randomize a proportion of the training data
## The corropution rate if we use random labels
if use_random_labels:
    corr_rate_list = [0, 0.2, 0.4, 0.6, 0.8, 0.9]
else:
    corr_rate_list = [0]
rank_on_train_data= True ## Calculate the rank of the training data or on random samples?
num_rank_val_samples = 100 ## The number of datapoints to calculate the rank on
num_epochs = 2 ## The number of epochs to train the network for
lr_list = [0.1, 0.01] ## Learning rates to try
dataset_list = ['KMNIST'] ## The datasets to train on ('KMNIST', 'MNIST', 'FashionMNIST') exists
per_batch = True ## If we want the training accuracy per batch
## The string that gets printed in the information file in each Experiment folder
extra_info_string = 'Just try and overfit the data, but should maybe have a learning rate scheduler then'

## For random labels it is easiest if datapoint_ind_store is a list [0,1,2,....,] 
## But otherwise it does not matter
datapoint_ind_store = [] ## The indices of the data points where to calculate the rank
for kkk in range(num_rank_val_samples):
  datapoint_ind_store.append(kkk)

## Perform the experiment for the desired parameters
for current_net in current_net_gen:
  for current_dataset in dataset_list:
    for lr in lr_list:
      for corr_rate in corr_rate_list:   
        train_loader, test_loader, train_loader_non_random = load_dataset(current_dataset)
        width_list = NET_dict[current_net]

        train_loader, data_point_store = get_random_loader(train_loader, corr_rate, datapoint_ind_store)
        
        ## Create the network
        my_net = Network(width_list)
        my_net = my_net.to(device)

        store_rank_val = {}
        store_rank_val[len(store_rank_val)] = calc_rank(my_net, 
                       num_rank_val_samples, data_point_store,
                       rank_on_train_data)
        

        my_net, train_info_dict = train_network(my_net, train_loader,
                                                test_loader, store_rank_val,
                                                per_batch, lr = lr,
                                                max_epochs = num_epochs, num_rank_val_samples = num_rank_val_samples,
                                                rank_on_train_data = rank_on_train_data,
                                                data_point_store = data_point_store)
        

        store_rank_val[len(store_rank_val)] = calc_rank(my_net, 
                       num_rank_val_samples, data_point_store, 
                       rank_on_train_data)
        

        ## We store all the information in this dictionary and later we pickle it for storage
        all_info_dict = {'Dataset': current_dataset, 'Current Net': current_net,
                        'Network width': NET_dict[current_net], 'lr': lr,
                        'Num epochs': num_epochs, 'Train info': train_info_dict,
                        'per_batch': per_batch, 'rank_val_bt': store_rank_val}#, 'num_dead_nodes_removed_opt': num_dead_nodes_removed_opt}
        

        ## Create the folder
        if not save_folder_created:
            save_folder = create_folder()
            write_info_txt(NET_dict, save_folder, extra_info_string)
            save_folder_created = True
        
        ## We save all info to a pickle
        my_file_save_name = [current_dataset, current_net, str(lr), str(num_epochs), str(corr_rate)]
        my_file_save_name = '_'.join(my_file_save_name)
        my_file_save_name = my_file_save_name + '.pickle'
        save_PATH = os.path.join(save_folder,my_file_save_name)
        
        with open(save_PATH, 'wb') as handle:
            pickle.dump(all_info_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        



Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw


Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw


Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw
Processing...
Done!
Epoch 0 Val Acc 0.7962999939918518 Train Acc 0.9109166860580444 Train Loss 0.19302503764629364
Epoch 1 Val Acc 0.8306999802589417 Train Acc 0.9386166930198669 Train Loss 0.30762821435928345
Epoch 0 Val Acc 0.682699978351593 Train Acc 0.8019999861717224 Train Loss 0.5623764395713806
Epoch 1 Val Acc 0.7804999947547913 Train Acc 0.9053500294685364 Train Loss 0.4041338860988617
Epoch 0 Val Acc 0.8183000087738037 Train Acc 0.930983304977417 Train Loss 0.29175350069999695
Epoch 1 Val Acc 0.8447999954223633 Train Acc 0.9484333395957947 Train Loss 0.39305630326271057
Epoch 0 Val Acc 0.7265999913215637 Train Acc 0.8559666872024536 Train Loss 0.29712724685668945
Epoch 1 Val Acc 0.8111000061035156 Train Acc 0.9280166625976562 Train Loss 0.2649633586406708
Epoch 0 Val Acc 0.8429999947547913 Train Acc 0.9372833371162415 Train Loss 0.28085610270500183
Epoch 1 Val Acc 0.8549000024795532 Train Acc 0.9462500

In [0]:
## The train accuracy is lower than the validation since I onl randomize the training loader.
import pickle
def plot_rank_during_training(experiment_folder, spec_word):

  pickle_folder = os.listdir(experiment_folder)
  pickle_list = [pickle_string for pickle_string in pickle_folder if spec_word in pickle_string]
  histogram_bins = [jjj for jjj in range(12)]
  ## I just plot every 5th histogram then?

  #for here
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots(len(pickle_list), 6, figsize=(30,len(pickle_list)*8))
  for current_row, pickle_info in enumerate(pickle_list):
    
    dict_info = pickle.load( open( os.path.join(experiment_folder, pickle_info ), "rb" ))
    rank_info = dict_info['rank_val_bt']
    plot_steps = [0, 5, 10, 15, 20]#[0,1,2,3] ## Hardcoded for 20 epochs
    #print(current_row)
    ax[current_row, 0].set_title(pickle_info)
    for idx, iii in enumerate(plot_steps):
      ax[current_row, idx].hist(rank_info[(iii)], bins = histogram_bins)
      ax[current_row, idx].set_xlabel('rank (epoch {})'.format(iii))
    train_acc = dict_info['Train info']['train_acc']
    val_acc = dict_info['Train info']['val_acc']
    ax[current_row, 5].plot(train_acc, label = 'train acc')
    ax[current_row, 5].plot(val_acc, label = 'val acc')
    ax[current_row, 5].legend()
    ax[current_row, 5].set_ylabel('Accuracy')
    ax[current_row, 5].set_xlabel('Epoch')


##20 not on train data and 27 on train data
## 30 should be on KMNIST
## 38 is on KMNIST where all samples are from shuffled labels
experiment_folder = 'Experiment 38'
spec_word = '0.1'
plot_rank_during_training(experiment_folder, spec_word)
